# パーティクルフィルタを使用した2次元の位置推定のシミュレーション
パーティクルフィルタで2次元の位置推定のシミュレートを行える．<br>
シミュレートするロボットの制御指令値と測距センサにはガウス分布に従うノイズがのる．<br>
青い線がロボットの軌道，緑の線が推定した軌道，星がランドマーク，黄色の点線が測距したランドマークへのレーザになっている．<br>

## クラスと関数
Init_value: 変数を定義する関数<br>
DefRobo: ロボットの移動や描画を行うクラス<br>
LandMark: ランドマークを描画するクラス<br>
Observe: 観測と観測したランドマークまでのレーザを描画<br>
MCL2: モンテカルロ位置推定を行うクラス<br>

## 設定
初期姿勢(0, -0.5, 0)<br>
制御指令値(1, 0.2)<br>
パーティクルの数 200個<br>
センサの最大測定距離 5m<br>
サンプリング時間 1s<br>
センサの測定値の標準偏差 0.5m<br>
速度指令値の標準偏差 0.2m/s<br>
角速度指令値の標準偏差 5degree/s<br>

In [1]:
import numpy as np
import math
import random 
import copy
import scipy.stats
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
from IPython import display

In [3]:
def Init_value():
    global INIT_X, INIT_Y, INIT_THETA, DT, END_TIME, LANDMARK_POSITION, MAX_RANGE, SENSOR_NOISE_STD, VEL_NOISE_STD, W_NOISE_STD, PARTICLE_NUM, INIT_VEL, INIT_W, NOISE_ODOM, DISTANCE_NOISE, ANGLE_NOISE
    '''初期姿勢'''
    INIT_X = 0
    INIT_Y = -5
    INIT_THETA = 0

    '''制御指令値'''
    INIT_VEL = 1
    INIT_W = 0.2
    
    '''サンプリングタイムと終了時間'''
    DT = 1
    END_TIME = 60
    
    '''センサの最大測定距離'''
    MAX_RANGE = 5

    '''パーティクルの数'''
    PARTICLE_NUM = 200
    
    '''シミュレートするロボットのノイズの標準偏差'''
    SENSOR_NOISE_STD = 0.5
    VEL_NOISE_STD = 0.2
    W_NOISE_STD = math.radians(5)
    
    '''パーティクルフィルタで使用する分散と標準偏差'''
    NOISE_ODOM = [[0.2, 0.001], 
                  [0.001, math.radians(5)]]
    DISTANCE_NOISE = 0.5
    ANGLE_NOISE = 0.01
    
    '''ランドマークの位置'''
    LANDMARK_POSITION = [[5.4, 5.5], [0, 6], [-3, -2], [1, 1], [-4.5, 5.5], [5, -10], [-12, 0]]

In [4]:
class DefRobo:
    '''独立対向二輪ロボット'''
    
    def __init__(self, name, pos, delta_time):
        self.name = name
        self.path_color = "blue"
        self.pos = np.asarray([[0], [0], [0]])
        self.pos = pos
        self.x_list = []
        self.y_list = []
        self.dt = delta_time

        self.x_list.append(self.pos[0][0])
        self.y_list.append(self.pos[1][0])

    def StateTransition(self, w, theta, u):
        '''状態遷移'''

        if w > 1e-5:
            A = np.asarray([[1/w * (math.sin(theta + w*self.dt) - math.sin(theta)), 0], 
                            [1/w * (-math.cos(theta + w*self.dt) + math.cos(theta)), 0], 
                            [0, self.dt]])
        else:
            A = np.asarray([[math.cos(theta), 0],
                            [math.sin(theta), 0],
                            [0, 1]]) * self.dt

        pos = A.dot(u)

        return pos
    
    def SetPathColor(self, color):
        '''軌道の色を設定'''
        self.path_color = color
        
    def GetPos(self, vel, w, noise_flag):
        '''ロボットを移動させ新しい姿勢を計算する'''
        rad_theta = self.pos[2][0]

        if noise_flag:
            u = np.asarray([[vel + np.random.normal(0, VEL_NOISE_STD)], 
                            [w + np.random.normal(0, W_NOISE_STD)]])
        else:
            u = np.asarray([[vel], 
                            [w]])
        
        self.pos = self.pos + self.StateTransition(w, rad_theta, u)
        self.pos[2][0] = self.pos[2][0] % (2 * math.pi)

        return self.pos.flatten()
    
    def DrawRobo(self, x, y, theta, plot_axes):
        '''ロボットを描画する'''
        circle = patches.Circle(xy=(x, y), radius=0.5, facecolor ='None', edgecolor='g')
        line = plt.plot([x, x+0.5*math.cos(theta)], [y, y+0.5*math.sin(theta)], color = "green")
        diagram = plot_axes.add_artist(circle)

        return line + [diagram]
    
    def DrawPath(self, x, y):
        '''ロボットの軌道を描画'''
        self.x_list.append(x)
        self.y_list.append(y)
        
        return plt.plot(self.x_list, self.y_list, color = self.path_color)

In [5]:
class LandMark:
    '''ランドマークの設定'''
    
    def __init__(self):
        self.num = 0

    def DrawLandMark(self, x, y):
        '''ランドマークの描画'''
        landmark = plt.plot(x, y, marker = "*", color = "red", markersize = 16)
        return landmark

In [6]:
class Observe:
    '''センサによる観測'''
    def __init__(self, max_range):
        self.l = 0
        self.phi = 0
        self.MAX_RANGE = max_range

    def DrawBeam(self, h, x, y, theta):
        '''測距できたランドマークまでのレーザを描画'''
        if (h[0] <= MAX_RANGE) and (h[0] >= 0):
            return plt.plot([x, x+h[0]*math.cos(h[1] + theta)], [y, y+h[0]*math.sin(h[1] + theta)], "--", color = "yellow")
        else:
            return []

    def Observation(self, x, y, theta, mx, my, Noise):
        '''観測方程式による計算'''
        if Noise:
            self.l = math.sqrt((mx - x)**2 + (my - y)**2) + np.random.normal(0, SENSOR_NOISE_STD)
            self.phi = math.atan2(my - y, mx - x) - theta
        else:
            self.l = math.sqrt((mx - x)**2 + (my - y)**2)
            self.phi = math.atan2(my - y, mx - x) - theta
            
        if self.phi < 0:
            self.phi += 2*math.pi

        if self.l > MAX_RANGE:
            self.l = -1
            self.phi = 0
            
        return self.l, self.phi

In [9]:
class MCL2(DefRobo, Observe):
    '''モンテカルロ位置推定'''

    def __init__(self, init_pose, delta_time, noise_odom, distance_noise, angle_noise, particle_num):
        
        self.particle_num = particle_num
        self.dt = delta_time
        self.noise_odom = noise_odom
        self.distance_noise = distance_noise
        self.angle_noise = angle_noise

        self.particle_x = [init_pose[0][0]] * self.particle_num
        self.particle_y = [init_pose[1][0]] * self.particle_num
        self.particle_theta = [init_pose[2][0]] * self.particle_num
        self.weight = [1/self.particle_num] * self.particle_num
        
        self.x_list = []
        self.y_list = []
        
        self.SetPathColor("green")

    def PredictMotion(self, axes_plot, vel, w):
        '''予測ステップ'''
        particle = []
        est_x = [0] * self.particle_num
        est_y = [0] * self.particle_num
        est_theta = [0] * self.particle_num
        
        for i in range(self.particle_num):
            est_x[i], est_y[i], est_theta[i] = self.UpdateParticlePosition(self.particle_x[i], self.particle_y[i], self.particle_theta[i], vel, w, self.noise_odom)
            particle.append(axes_plot.quiver(est_x[i], est_y[i], math.cos(est_theta[i]), math.sin(est_theta[i]), alpha=0.3))
            
        self.particle_x = copy.deepcopy(est_x[:])
        self.particle_y = copy.deepcopy(est_y[:])
        self.particle_theta = copy.deepcopy(est_theta[:])

        return particle

    def UpdateParticlePosition(self, x, y, theta, vel, w, noise_odom):
        '''パーティクルを状態遷移'''
        particle_pos = np.asarray([[x],[y],[theta]])
        
        u_d = np.asarray([[vel + np.random.normal(0, noise_odom[0][0]) * math.sqrt(abs(vel)/self.dt) + np.random.normal(0, noise_odom[0][1]) * math.sqrt(abs(w)/self.dt)],
                          [w + np.random.normal(0, noise_odom[1][0]) * math.sqrt(abs(vel)/self.dt) + np.random.normal(0, noise_odom[1][1]) * math.sqrt(abs(w)/self.dt)]])

        new_pos = particle_pos + self.StateTransition(w, theta, u_d)
        new_pos[2][0] = new_pos[2][0] % (2 * math.pi)

        return new_pos.flatten()
    
    def WeightUpdate(self, observe_data):
        '''観測によりパーティクルの重みを更新'''
        for i in range(self.particle_num):
            for j in range(len(observe_data)):
                if observe_data[j][0] >= 0:
                    obs_pole = observe_data[j]
                    est_particle_pos = self.Observation(self.particle_x[i], self.particle_y[i], self.particle_theta[i], LANDMARK_POSITION[j][0], LANDMARK_POSITION[j][1], Noise=False)

                    Q = np.asarray([[(self.distance_noise * est_particle_pos[0]) ** 2,   0],
                                    [0,                              self.angle_noise ** 2]])
                    self.weight[i] *= scipy.stats.multivariate_normal(mean=est_particle_pos, cov=Q).pdf(obs_pole)
        self.weight = np.array(self.weight) + 1e-300
        eta = sum(self.weight)
        
        max_likelihood_x = self.particle_x[np.argmax(self.weight)]
        max_likelihood_y = self.particle_y[np.argmax(self.weight)]
        self.weight = np.array(self.weight) / eta
        
        path_draw = self.DrawPath(max_likelihood_x, max_likelihood_y)
        
        return path_draw
        
    def SimpleResampling(self):
        '''単純なリサンプリング'''
        index = np.array(range(0, self.particle_num))
        sample_particle_index = np.random.choice(a=index, size=self.particle_num, p=self.weight)
        for i, selected_index in enumerate(sample_particle_index):
            self.particle_x[i] = self.particle_x[selected_index]
            self.particle_y[i] = self.particle_y[selected_index]
            self.particle_theta[i] = self.particle_theta[selected_index]
            self.weight[i] = self.weight[selected_index]
        
        self.weight = np.array(self.weight) + 1e-300
        eta = sum(self.weight)
        self.weight = np.array(self.weight) / eta


        

In [10]:
def main():
    Init_value()

    fig = plt.figure(figsize=(6,6))
    ax = plt.axes()
    plt.xlim(-15, 15)
    plt.ylim(-15, 15)

    frames = []
    beam = list()
    obs_plot = list()

    noised_robot = DefRobo("sim_robot", np.asarray([[INIT_X], [INIT_Y], [INIT_THETA]]), DT)
    landmark = LandMark()
    observe = Observe(MAX_RANGE)
    mcl = MCL2([[INIT_X], [INIT_Y], [INIT_THETA]], DT, NOISE_ODOM, DISTANCE_NOISE, ANGLE_NOISE, PARTICLE_NUM)

    #初期値の描画
    for j in range(len(LANDMARK_POSITION)):
            landmark_plot = landmark.DrawLandMark(LANDMARK_POSITION[j][0], LANDMARK_POSITION[j][1])
    frames.append(noised_robot.DrawPath(INIT_X, INIT_Y) + noised_robot.DrawRobo(INIT_X, INIT_Y, INIT_THETA, ax) + landmark_plot)

    #動画の作成
    for i in range(int(END_TIME/DT)):
        #ノイズを乗せた姿勢を取得
        noised_x, noised_y, noised_theta = noised_robot.GetPos(INIT_VEL, INIT_W, noise_flag=True)

        #ノイズを乗せたロボットの描画
        noised_path = noised_robot.DrawPath(noised_x, noised_y)
        noised_defrobo = noised_robot.DrawRobo(noised_x, noised_y, noised_theta, ax)
        
        #ランドマークの観測で使用する配列の初期化
        obs_plot = []
        observation_data = []
        #ランドマークの観測
        for j in range(len(LANDMARK_POSITION)):
            observe_data = observe.Observation(noised_x, noised_y, noised_theta, LANDMARK_POSITION[j][0], LANDMARK_POSITION[j][1], Noise=True)
            observation_data.append(observe_data)
            beam = observe.DrawBeam(observe_data, noised_x, noised_y, noised_theta)
            obs_plot = obs_plot + beam + landmark_plot


        '''パーティクルフィルタの処理'''
        #予測
        particles = mcl.PredictMotion(ax, INIT_VEL, INIT_W)
        
        #観測更新
        est_path = mcl.WeightUpdate(observation_data)  
        
        #リサンプリング
        mcl.SimpleResampling()
        
        #フレームの追加
        frames.append(noised_path + noised_defrobo + obs_plot + particles + est_path)
        

    plt.grid()
    ani = animation.ArtistAnimation(fig, frames, interval=100)
    #ani.save("movie/sim2.gif", writer="imagemagick") #アニメーションをgifでセーブ
    html = display.HTML(ani.to_jshtml())
    display.display(html)
    plt.close()

if __name__ == '__main__':
    main()

## 実行例
github上でgifが表示されない場合はリンクを踏んでください．<br>
[![sim2](movie/sim.gif)](movie/sim.gif) [![sim2](movie/sim2.gif)](movie/sim2.gif)
